In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from time import strftime
from time import gmtime

%matplotlib inline
from IPython.display import set_matplotlib_formats
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

matplotlib.rc('font', family = 'AppleGothic')
matplotlib.rc('axes', unicode_minus = False)
set_matplotlib_formats('retina')

# 1. Loading and cleaning the data

### 1.1. Loading Data

In [2]:
data = pd.read_csv('data/data.csv')
print(data.shape)
data.head()

datas = data.copy()

(434244, 11)


### 1.2. Merging with other datas 
- inappevent_category with funnel data
- inappevent_label with prudocut category data
- Clarifying funnel categories by merging inappevent_category data with funnel category data.

In [3]:
def funnelarrangement():
    data['viewid'] = data.inappevent_category.str.split('.').str[0]
    
    # Merging with Funnel data to clarify the funnel from 'inappevent_category'
    funnel = pd.read_csv('data/funnel.csv')
    funnel = funnel[['viewid', 'funnel_name']]
    df = data.merge(funnel, on = 'viewid', how = 'left') 
    
    # Merging with Category data to clarify the products from 'inappevent_label'

    category = pd.read_csv('data/category.csv')[['categoryid', 'category']]
    category.rename(columns = {'categoryid': 'inappevent_label'}, inplace = True)
    df = df.merge(category, on = 'inappevent_label', how = 'left')
    df.rename(columns = {'category': 'product_category'}, inplace = True)
    
    return df


datas = funnelarrangement()
datas


,n_user_id,os_version,source,channel,event_category,isfirst_activity,eventdatetime,campaign_keyword,inappevent_category,inappevent_label,isfirstgoal_activity,viewid,funnel_name,product_category
0,0,iOS11.4.1,unattributed,unattributed,goal,False,2018-09-28T00:00:00+09:00,NaN,home.view,0,False,home,home,NaN
1,1,Android7.0,unattributed,unattributed,goal,False,2018-09-28T00:00:00+09:00,NaN,gig_detail.view,41201,False,gig_detail,product,document
2,2,iOS12.0,unattributed,unattributed,goal,False,2018-09-28T00:00:00+09:00,NaN,inbox_detail.view,0,False,inbox_detail,inbox,NaN
3,3,iOS11.4.1,NaN,NaN,foreground,NaN,2018-09-28T00:00:01+09:00,NaN,NaN,0,NaN,NaN,NaN,NaN
4,4,iOS11.4.1,viral,WEB,goal,False,2018-09-28T00:00:02+09:00,NaN,buyer_order_track.view,0,False,buyer_order_track,order_management,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434239,9751,Android8.0.0,stranger,google,goal,False,2018-09-29T23:59:52+09:00,NaN,gig_detail.view,104,False,gig_detail,product,design
434240,4098,Android8.0.0,unattributed,unattributed,goal,False,2018-09-29T23:59:53+09:00,NaN,home.view,0,False,home,home,NaN
434241,9751,Android8.0.0,stranger,google,goal,False,2018-09-29T23:59:53+09:00,NaN,category_gig.view,104,False,category_gig,category,design
434242,4098,Android8.0.0,NaN,NaN,exit,NaN,2018-09-29T23:59:54+09:00,NaN,NaN,0,NaN,NaN,NaN,NaN


### 1.2. Arranging Features - os_version, channel, datetime


In [4]:
def features():
    
    ## OS_version preprocessing - Extracting OS name only from the os_version (ex. iOS11.4.1 -> iOS)
    datas.loc[data.os_version.str.contains('iOS'), 'os'] = 'iOS'
    datas.loc[data.os_version.str.contains('Android'), 'os'] = 'Android'
   
    # Time Preprocessing
    datas.eventdatetime = pd.to_datetime(datas.eventdatetime).dt.tz_convert(None)
    datas['time_h'] = datas.eventdatetime.dt.hour.apply(lambda x: '{:0>2d}'.format(x))
    datas['time_d'] = datas.eventdatetime.dt.day
    datas['time_dh'] = datas['time_d'].astype('str') + '_' + datas['time_h'].astype('str')
    
    return datas
datas = features()

def channel(row):
    if pd.isnull(row): return np.nan
    if 'google' in row: return 'google'
    elif 'naver' in row: return 'naver'
    elif 'daum' in row: return 'daum'
    elif 'apple' in row: return 'apple'
    else: return row

datas['channel'] = datas.channel.apply(channel)
datas

,n_user_id,os_version,source,channel,event_category,isfirst_activity,eventdatetime,campaign_keyword,inappevent_category,inappevent_label,isfirstgoal_activity,viewid,funnel_name,product_category,os,time_h,time_d,time_dh
0,0,iOS11.4.1,unattributed,unattributed,goal,False,2018-09-27 15:00:00,NaN,home.view,0,False,home,home,NaN,iOS,15,27,27_15
1,1,Android7.0,unattributed,unattributed,goal,False,2018-09-27 15:00:00,NaN,gig_detail.view,41201,False,gig_detail,product,document,Android,15,27,27_15
2,2,iOS12.0,unattributed,unattributed,goal,False,2018-09-27 15:00:00,NaN,inbox_detail.view,0,False,inbox_detail,inbox,NaN,iOS,15,27,27_15
3,3,iOS11.4.1,NaN,NaN,foreground,NaN,2018-09-27 15:00:01,NaN,NaN,0,NaN,NaN,NaN,NaN,iOS,15,27,27_15
4,4,iOS11.4.1,viral,WEB,goal,False,2018-09-27 15:00:02,NaN,buyer_order_track.view,0,False,buyer_order_track,order_management,NaN,iOS,15,27,27_15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434239,9751,Android8.0.0,stranger,google,goal,False,2018-09-29 14:59:52,NaN,gig_detail.view,104,False,gig_detail,product,design,Android,14,29,29_14
434240,4098,Android8.0.0,unattributed,unattributed,goal,False,2018-09-29 14:59:53,NaN,home.view,0,False,home,home,NaN,Android,14,29,29_14
434241,9751,Android8.0.0,stranger,google,goal,False,2018-09-29 14:59:53,NaN,category_gig.view,104,False,category_gig,category,design,Android,14,29,29_14
434242,4098,Android8.0.0,NaN,NaN,exit,NaN,2018-09-29 14:59:54,NaN,NaN,0,NaN,NaN,NaN,NaN,Android,14,29,29_14


### 1.3. Clarifying Session

In [5]:

def making_session():
    datas.sort_values(by = ['n_user_id', 'eventdatetime'], inplace = True)
    datas['eventdatetime_after'] = datas.groupby('n_user_id').eventdatetime.shift(-1)
    datas['page_duration'] = (datas['eventdatetime_after'] - datas['eventdatetime'])  # Finding page duration
    datas['page_duration'] = datas['page_duration'].map(lambda x: x.total_seconds()).fillna(0).astype(int) # Making it to seconds
    
    # Clarifying the session criteria, setting the condition as < 40 min or 0
    datas['is_out'] = (datas.page_duration == 0)|(datas.page_duration > 2400) # 60 seconds * 40 minutes
    datas.loc[datas.is_out == True, 'page_duration'] = 0
    
    # Making Session
    datas['session_idx_user'] = datas.groupby('n_user_id')['is_out'].cumsum().shift(1).fillna(0).astype(int)
    datas['session_idx'] = datas['is_out'].cumsum().shift(1).fillna(0).astype(int)

    # Making 0 for first session_idx 
    head = datas.groupby('n_user_id')['session_idx_user'].head(1).index
    datas.loc[head, 'session_idx_user'] = 0
    
    return datas.reset_index(drop = True)

datas = making_session()
datas[['n_user_id', 'eventdatetime', 'eventdatetime_after', 'page_duration', 'is_out', 'session_idx', 'session_idx_user']].head(100)

,n_user_id,eventdatetime,eventdatetime_after,page_duration,is_out,session_idx,session_idx_user
0,0,2018-09-27 15:00:00,2018-09-27 15:00:43,43,False,0,0
1,0,2018-09-27 15:00:43,2018-09-27 15:00:47,4,False,0,0
2,0,2018-09-27 15:00:47,2018-09-27 15:00:52,5,False,0,0
3,0,2018-09-27 15:00:52,2018-09-27 15:00:54,2,False,0,0
4,0,2018-09-27 15:00:54,2018-09-27 15:00:56,2,False,0,0
5,0,2018-09-27 15:00:56,2018-09-27 15:00:58,2,False,0,0
6,0,2018-09-27 15:00:58,2018-09-27 15:01:17,19,False,0,0
7,0,2018-09-27 15:01:17,2018-09-27 15:01:35,18,False,0,0
8,0,2018-09-27 15:01:35,2018-09-27 15:01:54,19,False,0,0
9,0,2018-09-27 15:01:54,2018-09-28 15:36:22,0,True,0,0


### 1.4. Google Analytics measurements

* Existing 48 hours data from Sep 27 15:00 to Sep 29 14:00 therefore daily comparison is irrelevant 
* This formula can be applied easily to other datas. 

In [6]:

def ganalytics(day):
    
    user_n = datas.loc[datas.time_d == day, 'n_user_id'].nunique()
    session_n = datas.loc[datas.time_d == day, 'session_idx'].nunique()
    pageview = len(datas.loc[(datas.inappevent_category.str.contains('.view') == True) & (datas.time_d == day), 'inappevent_category'])
    AvgSession = strftime("%H:%M:%S", gmtime(datas[datas.time_d == day].groupby('session_idx').page_duration.sum().mean()))
    newvisitor = 100 * len(datas[(datas.time_d == day)&(datas.isfirst_activity == True)]) / len(datas[datas.time_d == day])
    newcustomer = 100 * len(datas[(datas.time_d == day)&(datas.isfirstgoal_activity == True)]) / len(datas[datas.time_d == day])

    return print(f"""Sep {day}
    Sessions: {session_n}
    Users: {user_n}
    Pageviews: {pageview}
    Page/Session: {(pageview / session_n):.2f}
    % New Visitors: {newvisitor:.2f}%
    % New Customer: {newcustomer:.2f}%
    Avg. Session Duration: {AvgSession}
    """)

ganalytics(27) 
ganalytics(28) 
ganalytics(29)

Sep 27
    Sessions: 4513
    Users: 1904
    Pageviews: 24828
    Page/Session: 5.50
    % New Visitors: 14.13%
    % New Customer: 4.95%
    Avg. Session Duration: 00:04:14
    
Sep 28
    Sessions: 35004
    Users: 7237
    Pageviews: 176498
    Page/Session: 5.04
    % New Visitors: 8.71%
    % New Customer: 4.32%
    Avg. Session Duration: 00:05:54
    
Sep 29
    Sessions: 18895
    Users: 4829
    Pageviews: 94719
    Page/Session: 5.01
    % New Visitors: 9.32%
    % New Customer: 4.36%
    Avg. Session Duration: 00:05:09
    
